<a href="https://colab.research.google.com/github/hankiou/-Master-2-BI-Decision/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application BI: Aide à la décision
*Sujet choisi: Sujet 5 - Théorie du choix social*  
Trois profils de vote sont fournis. Le premier comporte 1000 électeurs et 9 candidats, les deux
autres comportent **10 000** électeurs et **12** candidats. Les données (accessibles vu la taille
uniquement en .csv) donnent **l’ordre de préférence** de chaque électeur pour les 12 candidats,
numérotés de 1 à 12.  
L'objectif est de programmer et de comparer les résultats obtenus par différents modes de scrutin possibles (vus en cours ou obtenus par recherche bibliographique).  
  
Nous effectuons les tests sur profil1, profil2, profil3 fournis en consigne, profil4 est un jeu de données extrait du TD *"examen 2"* fait en cours et nous sert de corpus de validation afin de comparer les résultats de nos implémentations avec les résultats obtenus en présentiel.


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
CORPUS_SIZE = 4 # How many files are executed

## Methods

### Tools

In [ ]:
def eliminate(matrix, candidate):
    shape = matrix.shape
    transpose = np.transpose(matrix)
    transpose = transpose[transpose != candidate]
    transpose = transpose.reshape(shape[1], shape[0] - 1)
    return np.transpose(transpose)

def getCandidates(matrix):
    return np.unique(matrix)

### Implementation

In [ ]:
# Selects elected candidate based on first preference
def relativeMajority(matrix, returnSize):
    return Counter(matrix[0]).most_common(returnSize)[0]

# Selects two winners for a first turn using relativeMajority()
# Eliminates every other candidate then processes a second turn among remaining candidates
def relativeMajorityT2(matrix):
    candidates = getCandidates(matrix)
    winners = [Counter(matrix[0]).most_common(2)[0][0], Counter(matrix[0]).most_common(2)[1][0]]
    eliminated = set(candidates) - set(winners)
    new_matrix = np.copy(matrix)
    for loser in eliminated:
        new_matrix[(new_matrix == loser)] = -1
        new_matrix = eliminate(new_matrix, -1)
    return Counter(new_matrix[0]).most_common(1)[0]
    
# Selects elected candidate based on borda method
# Score is calculated by adding (n - preference_order) for each voter
def borda(matrix):
    n = matrix.shape[0]
    scores = np.zeros(n).astype(np.int)
    voters = matrix.shape[1]
    for i in range (0, voters-1):
        for j in range (0, n-1):
            scores[matrix[j][i] -1] += (n - j)
    return scores.argmax()+1, scores.max()

def condorcet():
    #TODO
    return 0

def kemenyYoung():
    #TODO
    return 0

def coombs(matrix):
    n = matrix.shape[0] -1
    new_matrix = np.copy(matrix)
    for i in range (0, n-1):
        unpopular =  Counter(new_matrix[new_matrix.shape[0] - 1]).most_common(1)[0][0]
        new_matrix = eliminate(new_matrix, unpopular)
    return Counter(new_matrix[0]).most_common(1)[0]

def alternative():
    #TODO
    return 0

## Data

In [ ]:
# Load Data
# data/choix_social/

matrices = [None] * CORPUS_SIZE # Stores matrix conversions of each file
for i in range (0, CORPUS_SIZE):
    filename = 'data/choix_social/profil'+ str(i+1) + '.csv'
    data = pd.read_csv(filename, header=None)
    matrices[i] = data.to_numpy()
    print(matrices[i].shape)

(9, 1000)
(12, 10000)
(12, 10000)
(4, 100)


*TODO: Commentaire sur la data*

In [ ]:
# DEV SANDBOX


## Execution

In [ ]:
#Relative Majority 1 turn
print("Relative Majority 1 turn:")
for i in range (0, CORPUS_SIZE):
    result = relativeMajority(matrices[i], 1)
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

#Relative Majority 2 turns
print("\nRelative Majority 2 turns:")
for i in range (0, CORPUS_SIZE):
    result = relativeMajorityT2(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")


# Borda
print("\nBorda :")
for i in range (0, CORPUS_SIZE):
    result = borda(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + "pts")

# Coombs
print("\nCoombs :")
for i in range (0, CORPUS_SIZE):
    result = coombs(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix reportées (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")




Relative Majority 1 turn:
Profil1: Candidat 4: 293 voix (29.30 %)
Profil2: Candidat 10: 1828 voix (18.28 %)
Profil3: Candidat 7: 1498 voix (14.98 %)
Profil4: Candidat 1: 35 voix (35.00 %)

Relative Majority 2 turns:
Profil1: Candidat 4: 564 voix (56.40 %)
Profil2: Candidat 5: 6366 voix (63.66 %)
Profil3: Candidat 7: 5078 voix (50.78 %)
Profil4: Candidat 4: 65 voix (65.00 %)

Borda :
Profil1: Candidat 5: 6451pts
Profil2: Candidat 7: 95904pts
Profil3: Candidat 6: 80155pts
Profil4: Candidat 3: 306pts

Coombs :
Profil1: Candidat 5: 524 voix reportées (52.40 %)
Profil2: Candidat 7: 5398 voix reportées (53.98 %)
Profil3: Candidat 11: 5712 voix reportées (57.12 %)
Profil4: Candidat 3: 75 voix reportées (75.00 %)


## Interprétation des Résultats
### Profil 1
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|
|:-|:-|:-|:-|:-|:-|:-|
|**4** (293 - *29.30%*)|**4** (564 - *56.40%*)|-|**5** (6,451 pts)|**5** (524 - *52.40%*)|-|-|
### Profil 2
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|
|:-|:-|:-|:-|:-|:-|:-|
|**10** (1828 - *18.28%*)|**5** (6366 - *63.66%*)|-|**7** (95,904 pts)|**7** (5398 - *53.98%*)|-|-|
### Profil 3
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|
|:-|:-|:-|:-|:-|:-|:-|
|**7** (1498 - *14.97%*)|**7** (5078 - *50.78%*)|-|**6** (80,155 pts)|**5** (5712 - *57.12%*)|-|-|